# Sun Access Notebook

## 0. Initialization

### 0.1. Importing libraries

In [1]:
import EN_17037_Recipes as enr 
import os
import trimesh as tm 
import pyvista as pv
import ladybug as lb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


### 0.2. Setting global variables

In [2]:
# directory variables
cwd = os.getcwd()
src = os.path.dirname(cwd)

# project name
project_name = 'EN_17037_sun_access'

# create the material dictionary
material_dict = {
    "material_plastic": enr.material_plastic,
    "material_glass": enr.material_glass
}

# load the surface information table
surface_df = pd.read_csv("sa_file_table.csv")
surface_df

,surface_name,surface_type,file_path,material_name
0,wall,0,ref_room_walls.obj,material_plastic
1,window,5,ref_room_window.obj,material_glass
2,context_buildings,0,test_context_01_buildings.obj,material_plastic


### 0.3. Prepare Analysis Information

In [3]:
# prepare grid points
grid_path = os.path.join(src, "data", "point_grid_sunacc.csv")
grid_array = np.genfromtxt(grid_path, delimiter=",")
pnts = [tuple(pnt) for pnt in grid_array] if len(grid_array.shape) == 2 else [tuple(grid_array)]
up_vecs = [(0, 0, 1)] * len(pnts)

# initiate sunpath
sp = lb.sunpath.Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
hoy_range = np.arange(0, 8760, 1)
for i in hoy_range:
    # compute the sun object
    sun = sp.calculate_sun_from_hoy(i, is_solar_time=True)
    # extract the sun vector
    sun_vector = sun.sun_vector.to_array()

    hoys.append(i)
    sun_vectors.append(sun_vector)

In [10]:
sun_vectors = np.array(sun_vectors)
hoys = np.array(hoys)

(array([ 0,  1,  2, ..., 21, 22, 23]),
 array([  0,   0,   0, ..., 364, 364, 364]))

In [11]:
np.savetxt("../data/sun_vectors.csv", np.c_[sun_vectors,hoys % 24, (hoys / 24).astype(int)], delimiter=",")